In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append(os.path.realpath('..'))

In [ ]:
import cache_magic
import numpy as np
import scipy as sp
import pandas as pd
import plotly.express as ex
import plotly.graph_objects as go
import concurrent.futures
from copy import deepcopy

from tracking_v2.target import ConstantVelocityTarget, SingleTurnTarget
from tracking_v2.kalman import LinearKalmanFilter, CoordinatedTurn
from tracking_v2.motion import ConstantVelocityModel, ConstantAccelerationModel, SingerAccelerationModel
from tracking_v2.sensor import GeometricSensor
from tracking_v2.evaluation import Runner, evaluate_nees, plot_error, plot_3d, plot_nees, evaluate_runner

from tracking.util import to_df

In [ ]:
def _run_cv(target, sensor, q=1, m=100, n=500):
    motion = ConstantVelocityModel(q)
    kf = LinearKalmanFilter(motion, [[1, 0, 0, 0, 0, 0],
                                     [0, 1, 0, 0, 0, 0],
                                     [0, 0, 1, 0, 0, 0]])

    r = Runner(target, sensor, kf)
    r.run_many(m, n)
    return r

In [ ]:
def _run_ca(target, sensor, q=1, m=100, n=500):
    motion = ConstantAccelerationModel(q)
    kf = LinearKalmanFilter(motion, [[1, 0, 0, 0, 0, 0, 0, 0, 0],
                                     [0, 1, 0, 0, 0, 0, 0, 0, 0],
                                     [0, 0, 1, 0, 0, 0, 0, 0, 0]])

    r = Runner(target, sensor, kf)
    r.run_many(m, n)
    return r

In [ ]:
def _run_ct(target, sensor, q_xyz=1, q_omega=.0001, m=100, n=500):
    kf = CoordinatedTurn([q_xyz, q_xyz, q_xyz, q_omega])
    r = Runner(target, sensor, kf)
    r.run_many(m, n)
    return r

In [ ]:
def _metrics(runner):
    nees = evaluate_nees(runner.many_x_hat[:, 100:, :3, :], runner.many_P_hat[:, 100:, :3, :3], runner.truth[101:, :3])
    err  = np.linalg.norm(runner.many_x_hat[:,100:,:3,0] - runner.truth[101:,:3], axis=2)
    
    mean_nees = nees.scores.mean(axis=0)
    q975_nees = np.quantile(nees.scores, .975, axis=0)

    mean_err = err.mean(axis=0)
    q975_err = np.quantile(err, .975, axis=0)

    part = np.asarray((mean_nees, q975_nees, mean_err, q975_err)).T
    return to_df(part, columns=['nees_mean', 'nees_q975', 'err_mean', 'err_q975'])

def _run_models(target):
    sensor = GeometricSensor()

    data = []
    for Q in [0, .001, .01, .1, .5, 1, 1.5, 2, 2.5, 5, 10, 20]:
        m = _metrics(_run_cv(target, sensor, Q))
        m['Q'] = str(Q)
        m['model'] = 'cv'
        data.append(m)

        m = _metrics(_run_ca(target, sensor, Q))
        m['Q'] = str(Q)
        m['model'] = 'ca'
        data.append(m)

        m = _metrics(_run_ct(target, sensor, Q, Q * .0001))
        m['Q'] = str(Q)
        m['model'] = 'ct'
        data.append(m)

    return pd.concat(data)

# Target: Constant Velocity

In [ ]:
%cache cv_data = _run_models(ConstantVelocityTarget())

In [ ]:
fig = ex.box(cv_data.melt(['Q', 'model'], ['nees_mean', 'nees_q975', 'err_mean', 'err_q975'], 'metric', 'value'),
             x='Q', y='value', color='metric', facet_row='model')

ci = sp.stats.chi2.ppf([0.025, 0.975], 100 * 3) / 100
fig.add_hline(y=ci[0], line_width=.5, line_dash="dash", line_color="red")
fig.add_hline(y=ci[1], line_width=.5, line_dash="dash", line_color="red")

fig.update_layout(height=900, yaxis_range=[-1,20])
fig.show()

# Target: Single Turn

In [ ]:
%cache st_data = _run_models(SingleTurnTarget())

In [ ]:
fig = ex.box(st_data.melt(['Q', 'model'], ['nees_mean', 'nees_q975', 'err_mean', 'err_q975'], 'metric', 'value'),
             x='Q', y='value', color='metric', facet_row='model')

ci = sp.stats.chi2.ppf([0.025, 0.975], 100 * 3) / 100
fig.add_hline(y=ci[0], line_width=.5, line_dash="dash", line_color="red")
fig.add_hline(y=ci[1], line_width=.5, line_dash="dash", line_color="red")

fig.update_layout(height=900, yaxis_range=[-1,20])
fig.show()

In [ ]:
r = _run_ct(SingleTurnTarget(), GeometricSensor(), 1.5, .00015)

In [ ]:
plot_nees(evaluate_runner(r).position_nees)